In [283]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Read in and clean data

In [278]:
dropRows = [55, 76, 78, 81, 410, 481, 486, 490, 645, 653, 657, 693,
            697, 840, 827, 830, 842, 844, 949, 950, 1059, 1063, 1077, 1079, 1081]
#original data set had nonsense rows

superheroData = \
    pd.read_excel("Superhero data.xlsx").\
    drop(dropRows).\
    drop(["full_name"], axis = 1).\
    dropna(how = "all").\
    reset_index()#.\
    #drop(["index"], axis = 1)

#Fix format of height and weight variables:
columnLength = 1408

for i in range(0, columnLength - 1):
        #height data is of the form "6'8 â€¢ 203 cm" which would be very difficult to work with
        #hence I'm extracting just the value in centimeters and converting to a float
        #similar for weight

        if not(str(superheroData["height"][i]) == "nan"):
            try:
                superheroData["height"][i] = float(str(superheroData["height"][i]).split()[2])
            except:
                superheroData["height"][i] = str(superheroData["height"][i]).replace(',', '')
                superheroData["height"][i] = float(superheroData["height"][i].split()[2])

        if not(str(superheroData["weight"][i]) == "nan"):
            try:
                superheroData["weight"][i] = float(str(superheroData["weight"][i]).split()[3])
            except:
                superheroData["weight"][i] = str(superheroData["weight"][i]).replace(',', '')
                superheroData["weight"][i] = float(superheroData["weight"][i].split()[3])

superheroData

C:\Users\mbles\AppData\Local\Temp\ipykernel_9100\1170556366.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  superheroData["height"][i] = float(str(superheroData["height"][i]).split()[2])


,index,name,real_name,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.",85,30,60,60,40,70,"['Super Speed', 'Super Strength']",...,0,0,0,0,0,1,0,0,0,1
1,1,514A (Gotham),Bruce Wayne,100,20,30,50,35,100,"['Durability', 'Reflexes', 'Super Strength']",...,0,0,0,0,1,0,1,0,0,1
2,2,A-Bomb,Richard Milhouse Jones,80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",...,0,1,0,0,1,1,1,1,1,1
3,3,Aa,Aa,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",...,0,0,0,0,0,0,0,0,0,0
4,4,Aaron Cash,Aaron Cash,80,10,25,40,30,50,"['weapon based Powers', 'Weapons Master']",...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,1428,Zane,Zane,100,35,50,60,100,100,"['Agility', 'Animal Control', 'Cold Resistance...",...,1,0,1,1,1,0,1,1,1,1
1404,1429,Zatanna,Zatanna Zatara,90,10,25,30,100,55,"['Cryokinesis', 'Fire Control', 'Magic', 'Prob...",...,0,0,0,0,0,0,0,0,0,0
1405,1430,Zoom (New 52),Hunter Zolomon,95,50,100,75,100,80,"['Accelerated Healing', 'Agility', 'Durability...",...,0,1,0,1,1,1,1,1,1,1
1406,1431,Zoom,Hunter Zolomon,75,10,100,30,100,30,"['Intangibility', 'Super Speed', 'Time Manipul...",...,0,0,0,0,0,1,0,0,0,0


In [279]:
superheroData.isnull().sum()

index                   0
name                    0
real_name             137
intelligence_score     16
strength_score         16
                     ... 
has_super_speed        78
has_durability         78
has_stamina            78
has_agility            78
has_super_strength     78
Length: 68, dtype: int64

Impute missing data values:

In [292]:
quantitativeVariables = superheroData.iloc[:, [3, 4, 5, 6, 7, 8, 14, 15]]

temp = [None]*(68-16)
for i in range(16 - 16, 68):
    temp[i] = i + 17

categoricalVariables = superheroData.iloc[:, [0, 1, 2, 9, 10, 11, 12, 13] + temp]

knnImputer = KNNImputer(n_neighbors = 10)

quantitativeVariablesFilled = \
    pd.DataFrame(knnImputer.fit_transform(quantitativeVariables)).\
    rename(columns = {0:"intelligence score", 1:"strength score",
                      2:"speed score", 3:"durability score",
                      4:"power score", 5: "combat score",
                      6:"height (cm)", 7:"weight (kg)" })

quantitativeVariablesFilled




,intelligence score,strength score,speed score,durability score,power score,combat score,height (cm),weight (kg)
0,85.0,30.0,60.0,60.0,40.0,70.0,165.7,76.7
1,100.0,20.0,30.0,50.0,35.0,100.0,183.4,83.4
2,80.0,100.0,80.0,100.0,100.0,80.0,203.0,441.0
3,80.0,50.0,55.0,45.0,100.0,55.0,836.1,73.3
4,80.0,10.0,25.0,40.0,30.0,50.0,171.5,96.9
...,...,...,...,...,...,...,...,...
1403,100.0,35.0,50.0,60.0,100.0,100.0,168.0,45.0
1404,90.0,10.0,25.0,30.0,100.0,55.0,170.0,57.0
1405,95.0,50.0,100.0,75.0,100.0,80.0,185.0,81.0
1406,75.0,10.0,100.0,30.0,100.0,30.0,185.0,81.0


In [311]:
temp = [i for i in range(18, 68)]

temp

categoricalVariables = superheroData.iloc[:, [0, 1, 2, 9, 10, 11, 12, 13] + temp]

categoricalVariables

,index,name,real_name,superpowers,creator,alignment,gender,type_race,has_electrokinesis,has_energy_constructs,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","['Super Speed', 'Super Strength']",Marvel Comics,Good,Male,Human,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1,514A (Gotham),Bruce Wayne,"['Durability', 'Reflexes', 'Super Strength']",DC Comics,NaN,Male,NaN,0,0,...,0,0,0,0,1,0,1,0,0,1
2,2,A-Bomb,Richard Milhouse Jones,"['Accelerated Healing', 'Agility', 'Berserk Mo...",Marvel Comics,Good,Male,Human,0,0,...,0,1,0,0,1,1,1,1,1,1
3,3,Aa,Aa,"['Energy Absorption', 'Energy Armor', 'Energy ...",DC Comics,Good,Male,Human,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,Aaron Cash,Aaron Cash,"['weapon based Powers', 'Weapons Master']",DC Comics,Good,Male,Human,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,1428,Zane,Zane,"['Agility', 'Animal Control', 'Cold Resistance...",NaN,Good,Male,Android,0,0,...,1,0,1,1,1,0,1,1,1,1
1404,1429,Zatanna,Zatanna Zatara,"['Cryokinesis', 'Fire Control', 'Magic', 'Prob...",DC Comics,Good,Female,Human,0,0,...,0,0,0,0,0,0,0,0,0,0
1405,1430,Zoom (New 52),Hunter Zolomon,"['Accelerated Healing', 'Agility', 'Durability...",DC Comics,Bad,Male,Metahuman,1,0,...,0,1,0,1,1,1,1,1,1,1
1406,1431,Zoom,Hunter Zolomon,"['Intangibility', 'Super Speed', 'Time Manipul...",DC Comics,Bad,Male,Metahuman,0,0,...,0,0,0,0,0,1,0,0,0,0


In [312]:
temp

[18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67]